## Base Models

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split

In [2]:
pos_final_test = pd.read_csv('./pos_final_test.csv.gz', compression='gzip')
pos_final_test_KNN = pd.read_csv('./pos_KNN_final_test.csv.gz', compression='gzip')
pos_final_missing_data_test = pd.read_csv('./pos_final_missing_data_test.csv.gz', compression='gzip')


In [58]:
model_cols = ~pos_final_test.columns.isin(['(\'subject_id\', \'\')', '(\'hadm_id\', \'\')', 'subject_id', 'hadm_id', 'Unnamed: 0', 'pos'])

hadm_features = pos_final_test.loc[:, model_cols]
hadm_target = pos_final_test.loc[:, 'pos']

# hadm_features = pos_final_missing_data_test.loc[:, model_cols]
# hadm_target = pos_final_missing_data_test.loc[:, 'pos']


In [24]:
# # Creatinine - 50912
# hadm_features = hadm_features.loc[:, ['(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]

# # Least Missingness
# # [51221, 51265, 51222, 51301, 51279, 51277, 51248, 51249, 51250]
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)']]

# # Least Missingness + Creatinine
# hadm_features = hadm_features.loc[:, ['(\'max\', 51221)',  '(\'above_max\', 51221)', '(\'mean\', 51221)',  '(\'abn_percent\', 51221)',
#                                         '(\'max\', 51265)',  '(\'above_max\', 51265)', '(\'mean\', 51265)',  '(\'abn_percent\', 51265)',
#                                         '(\'max\', 51222)',  '(\'above_max\', 51222)', '(\'mean\', 51222)',  '(\'abn_percent\', 51222)',
#                                         '(\'max\', 51301)',  '(\'above_max\', 51301)', '(\'mean\', 51301)',  '(\'abn_percent\', 51301)',
#                                         '(\'max\', 51279)',  '(\'above_max\', 51279)', '(\'mean\', 51279)',  '(\'abn_percent\', 51279)',
#                                         '(\'max\', 51277)',  '(\'above_max\', 51277)', '(\'mean\', 51277)',  '(\'abn_percent\', 51277)',
#                                         '(\'max\', 51248)',  '(\'above_max\', 51248)', '(\'mean\', 51248)',  '(\'abn_percent\', 51248)',
#                                         '(\'max\', 51249)',  '(\'above_max\', 51249)', '(\'mean\', 51249)',  '(\'abn_percent\', 51249)',
#                                         '(\'max\', 51250)',  '(\'above_max\', 51250)', '(\'mean\', 51250)',  '(\'abn_percent\', 51250)',
#                                      '(\'max\', 50912)',  '(\'above_max\', 50912)', '(\'mean\', 50912)',  '(\'abn_percent\', 50912)']]


In [60]:
X_train, X_test, y_train, y_test = train_test_split(hadm_features, hadm_target, stratify=hadm_target, random_state=0)


## Logistic Regression

In [27]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import GridSearchCV

In [62]:
%%time

# L2
params_log = [ {'C':np.logspace(-4, 3, 20)} ]
logReg = LogisticRegression(max_iter = 2000, class_weight = "balanced")

# # L1
# params_log = [ {'C':np.logspace(-6, -1, 20)} ]
# logReg = LogisticRegression(penalty = 'l1', max_iter = 2000, class_weight = "balanced", solver = 'liblinear')

grid_log = GridSearchCV(estimator = logReg, param_grid = params_log, cv = 5)
grid_log.fit(X_train, y_train)
grid_log.best_score_

/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)
/opt/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/_logistic.py:764: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-reg

CPU times: user 4min 17s, sys: 36.3 s, total: 4min 54s
Wall time: 50.7 s


0.8494693877551022

In [63]:
grid_log.best_params_

{'C': 0.01623776739188721}

In [64]:
list(zip(X_train.columns, grid_log.best_estimator_.coef_[0]))


[("('min', 50861)", -0.12697421603913533),
 ("('min', 50863)", 0.07712324606402488),
 ("('min', 50868)", 0.04196641948989999),
 ("('min', 50878)", -0.005493361763311199),
 ("('min', 50882)", -0.07323988215453303),
 ("('min', 50885)", -0.4266608024632236),
 ("('min', 50893)", 0.12851293427581617),
 ("('min', 50902)", 0.008724548273672996),
 ("('min', 50911)", 0.014616357013603124),
 ("('min', 50912)", 0.01746223824239281),
 ("('min', 50931)", -0.2469226873717972),
 ("('min', 50960)", 0.040718098151666465),
 ("('min', 50970)", 0.0009609037955210141),
 ("('min', 50971)", 0.014551005627579335),
 ("('min', 50983)", 0.23426543711214362),
 ("('min', 51003)", 0.04756380530975304),
 ("('min', 51006)", 0.1465466738332293),
 ("('min', 51221)", -0.03777470484865111),
 ("('min', 51222)", -0.13822031305724133),
 ("('min', 51237)", -0.03029260877099509),
 ("('min', 51248)", -0.019816484484636628),
 ("('min', 51249)", -0.37091242784564493),
 ("('min', 51250)", 0.08024571537498087),
 ("('min', 51265)",

In [41]:
from sklearn.metrics import accuracy_score, confusion_matrix, precision_score, recall_score, f1_score

In [65]:
pred = grid_log.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, pred) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, pred) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, pred)}\n")

Accuracy Score: 84.82%
_______________________________________________
Classification Report:	Precision Score: 36.67%
			Recall Score: 74.63%
			F1 score: 49.18%
_______________________________________________
Confusion Matrix: 
 [[1582  259]
 [  51  150]]



## Random Forest

In [43]:
from sklearn import ensemble
import sklearn.model_selection as ms

randomForest = ensemble.RandomForestClassifier(class_weight = "balanced")

In [61]:
%%time

grid_para_forest = {
    'criterion': ['gini', 'entropy'],
    'max_depth': range(1, 20),
    'n_estimators': range(10, 110, 10)
}

grid_search_forest = ms.GridSearchCV(randomForest, grid_para_forest, scoring='accuracy', cv=5, n_jobs=-1)
grid_search_forest.fit(X_train, y_train)

CPU times: user 12 s, sys: 649 ms, total: 12.7 s
Wall time: 4min 34s


GridSearchCV(cv=5, estimator=RandomForestClassifier(class_weight='balanced'),
             n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_depth': range(1, 20),
                         'n_estimators': range(10, 110, 10)},
             scoring='accuracy')

In [45]:
# Get most important features
tree_final = grid_search_forest.best_estimator_
feature_importance = tree_final.feature_importances_
feature_importance = list(zip(X_train.columns, feature_importance))

feature_importance.sort(key = lambda x: x[1], reverse = True)
feature_importance

[("('mean', 51006)", 0.04197747339413929),
 ("('max', 51006)", 0.03880490032009373),
 ("('min', 51006)", 0.033616799897440736),
 ("('mean', 50912)", 0.033396543064347646),
 ("('max', 50912)", 0.025399107540205957),
 ("('above_max', 51006)", 0.02539193601329139),
 ("('abn_percent', 51006)", 0.022264621884633275),
 ("('above_max', 50912)", 0.0222242965622723),
 ("('max', 50885)", 0.018847500811477215),
 ("('min', 51237)", 0.017719038784028925),
 ("('min', 51274)", 0.015248551613305178),
 ("('abn_percent', 50912)", 0.014442731495209868),
 ("('mean', 51249)", 0.014161904903982595),
 ("('mean', 51237)", 0.013965801337791867),
 ("('above_max', 50931)", 0.013694578666119362),
 ("('min', 51003)", 0.012936175540029076),
 ("('abn_percent', 51274)", 0.012442914443154795),
 ("('min', 50885)", 0.011995633359198328),
 ("('mean', 50885)", 0.011662220446188318),
 ("('max', 51237)", 0.011558944710344942),
 ("('min', 51277)", 0.011392344336605301),
 ("('mean', 51274)", 0.011053227213532694),
 ("('max', 

In [46]:
grid_search_forest.best_params_

{'criterion': 'entropy', 'max_depth': 12, 'n_estimators': 80}

In [47]:
grid_search_forest.best_score_

0.9204897959183673

In [48]:
print("The training error is: %.5f" % (grid_search_forest.score(X_train, y_train)))
print("The test     error is: %.5f" % (grid_search_forest.score(X_test, y_test)))

The training error is: 0.99347
The test     error is: 0.90304


In [49]:
rf_predictions = tree_final.predict(X_test)

print(f"Accuracy Score: {accuracy_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print("Classification Report:", end='')
print(f"\tPrecision Score: {precision_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tRecall Score: {recall_score(y_test, rf_predictions) * 100:.2f}%")
print(f"\t\t\tF1 score: {f1_score(y_test, rf_predictions) * 100:.2f}%")
print("_______________________________________________")
print(f"Confusion Matrix: \n {confusion_matrix(y_test, rf_predictions)}\n")

Accuracy Score: 90.30%
_______________________________________________
Classification Report:	Precision Score: 51.01%
			Recall Score: 37.81%
			F1 score: 43.43%
_______________________________________________
Confusion Matrix: 
 [[1768   73]
 [ 125   76]]

